### import libs

In [ ]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = t.device('cuda' if t.cuda.is_available() else 'cpu')

## defing model


In [ ]:
notes = get_notes()

Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/FF4.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/FF6epitaph_piano.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/FFIII_Edgar_And_Sabin_Piano.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/FF8_Shuffle_or_boogie_pc.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/Fyw_piano.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/OTD5YA.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/FFIXQuMarshP.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/Life_Stream.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/ViviinAlexandria.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/Final_Fantasy_Matouyas_Cave_Piano.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/Ff7-Cinco.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/Gold_Silver_Rival_Battle.mid
Parsing /content/drive/MyDrive

/usr/local/lib/python3.9/dist-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=6, channel=None, data=b'Pok\xe9mon: Elite Four (Piano)'>; getting generic Instrument
  warnings.warn(


Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/pkelite4.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/ultros.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/goldsaucer.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/sandy.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/tpirtsd-piano.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/roseofmay-piano.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/ultimafro.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/sobf.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/ff8-lfp.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/thoughts.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/traitor.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/tifap.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/redwings.mid
Parsing /content/drive/MyDrive/ML_Project/midi_son

/usr/local/lib/python3.9/dist-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Martin Rosok Copyright \xa9 1998'>; getting generic Instrument
  warnings.warn(


Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/ff7themep.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/mining.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/relmstheme-piano.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/lurk_in_dark.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/ff4-fight1.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/ff4_piano_collections-main_theme.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/thenightmarebegins.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/ff6shap.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/figaro.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/path_of_repentance.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/ff4pclov.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/rufus.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/ff4-town.mid


In [ ]:
n_vocab = len(set(notes))

In [ ]:
network_input, network_output = prepare_sequences(notes, n_vocab)

In [ ]:
model = create_network(network_input, n_vocab)

In [ ]:
train(model, network_input, network_output)

Epoch 1/200
  7/235 [..............................] - ETA: 2:40 - loss: 5.0485

KeyboardInterrupt: ignored

In [ ]:
def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in glob.glob("/content/drive/MyDrive/ML_Project/midi_songs_sample/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('/content/drive/MyDrive/ML_Project/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=10, batch_size=128, callbacks=callbacks_list)


In [ ]:
train_network()

Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/FF4.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/FF6epitaph_piano.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/FFIII_Edgar_And_Sabin_Piano.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/FF8_Shuffle_or_boogie_pc.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/Fyw_piano.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/OTD5YA.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/FFIXQuMarshP.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/Life_Stream.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/ViviinAlexandria.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/Final_Fantasy_Matouyas_Cave_Piano.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/Ff7-Cinco.mid
Parsing /content/drive/MyDrive/ML_Project/midi_songs_sample/Gold_Silver_Rival_Battle.mid
Parsing /content/drive/MyDrive

Epoch 1/10
235/235 [==============================] - 186s 760ms/step - loss: 5.4935
Epoch 2/10
235/235 [==============================] - 175s 741ms/step - loss: 4.6849
Epoch 3/10
235/235 [==============================] - 175s 745ms/step - loss: 4.5610
Epoch 4/10
235/235 [==============================] - 177s 752ms/step - loss: 4.5102
Epoch 5/10
235/235 [==============================] - 176s 747ms/step - loss: 4.4652
Epoch 6/10
235/235 [==============================] - 175s 744ms/step - loss: 4.4238
Epoch 7/10
235/235 [==============================] - 173s 737ms/step - loss: 4.3941
Epoch 8/10
235/235 [==============================] - 175s 744ms/step - loss: 4.3586
Epoch 9/10
235/235 [==============================] - 175s 744ms/step - loss: 4.3287
Epoch 10/10
235/235 [==============================] - 176s 748ms/step - loss: 4.2974


## predicting model

In [ ]:
def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('/content/drive/MyDrive/ML_Project/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

def prepare_sequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights('/content/weights-improvement-10-4.2974-bigger.hdf5') 

    return model

def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')

In [ ]:
print(generate())

None
